In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
import hashlib
from sklearn.externals import joblib
import numpy
from operator import itemgetter
from itertools import groupby
with open("processed_tweets.csv", "r") as fl:
    all_content = fl.read().split("\n")

all_content = [[w.replace("\"", "") for w in t.split(",")] for t in all_content]
tweets = [(t[0], t[1], t[2]) for t in all_content if len(t) > 4 and t[3].lower() =="tweet"]
bios = [(t[1], t[2]) for t in all_content if len(t) > 4 and t[3].lower() == "bio"]

# tweetsGrouped = [(k, len([x for x,y in g])) for k,g in groupby(tweets, key=itemgetter(0))]
# tweetsGrouped.sort(key=itemgetter(1), reverse=True)
# print(len(tweetsGrouped))
# tweetvectorizer = TfidfVectorizer(stop_words="english")
# tweetvectorizer.fit_transform(tweets)
# #joblib.dump(tweetvectorizer, "tweetvector")

biovectorizer = TfidfVectorizer(stop_words="english", max_features=1500)
biovectorizer.fit_transform([t[1] for t in bios])

#joblib.dump(biovectorizer, "biovector")

biovecs = [(t[0], biovectorizer.transform([t[1]])[0]) for t in bios]
print(biovecs[0][1])
with open("kv-data", "r") as ft:
    all_items = ft.read().split("\n")
    
all_items = [[w.replace("\"", "") for w in t.split(",")] for t in all_items]
all_item_descs = [t[5][len("description:"):] for t in all_items if len(t) >= 6]
descriptionvectorizer = TfidfVectorizer(stop_words='english', max_features=1500)
descriptionvectorizer.fit_transform(all_item_descs)
#hashlib.md5(t[2][len("name:"):].encode()).hexdigest())
all_items = [((hashlib.md5(t[2][len("name:"):].encode()).hexdigest()), 
             descriptionvectorizer.transform([t[5][len("description:"):]])[0]) 
             for t in all_items if len(t) >= 5]
print(all_items[0])




  (0, 350)	0.586572326689
  (0, 300)	0.586572326689
  (0, 111)	0.476236799689
  (0, 93)	0.291657884766


('e15ac260bca0119df0eccc5817d755dd', <1x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>)


In [41]:
print()
finalDims = (all_items[0][1].shape[1], 
             biovecs[0][1].shape[1])
finalMatrix = numpy.zeros(finalDims)
# print()

In [43]:
for bio in biovecs:
    for item in all_items:
        if bio[0] == item[0]:
            mat = item[1].T
            for i, e in enumerate(mat):
                if e[0][0] != 0.:
                    finalMatrix[i] += bio[1]
print(finalMatrix)


[[ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 ..., 
 [ 0.         0.7659985  0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]]


In [44]:
print(finalMatrix.shape)

(1500, 1500)


In [45]:
numpy.savetxt("consumeritemmatrix2.csv", finalMatrix)

In [48]:
cnt = 0
for i in finalMatrix:
    for j in i:
        if j == 0.:
            cnt+=1
print(cnt/float(1500*1500))

0.9512106666666666


In [47]:
print(1500*1500)

2250000
